In [1]:
import dotenv
import nest_asyncio

dotenv.load_dotenv()
nest_asyncio.apply()

In [2]:
import os
from atproto import AsyncClient

client = AsyncClient()
await client.login(os.environ["BSKY_USR"], os.environ["BSKY_PWD"])

/home/cole/school/Trust-and-Safety/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'default' attribute with value None was provided to the `Field()` function, which has no effect in the context it was used. 'default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


ProfileViewDetailed(did='did:plc:y3nd3tn2labpv3ykyzu6c44p', handle='alexiosfanclub.bsky.social', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:y3nd3tn2labpv3ykyzu6c44p/bafkreig2s7qacccfw3kvnqnbv7dsaymv3r7zaguvvzz5vakfxgolue5xdy@jpeg', banner=None, created_at='2025-10-30T17:41:54.451Z', description=None, display_name='', followers_count=0, follows_count=1, indexed_at='2025-10-30T17:41:54.451Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=0, pronouns=None, status=None, verification=None, viewer=ViewerState(activity_subscription=None, blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=Fal

In [3]:
prof = await client.get_profile("mantzarlis.com")

In [4]:
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
from atproto_client.models.app.bsky.actor.defs import ProfileView

async def fetch_all_following(handle: str | ProfileView, client: AsyncClient, batch_limit=100, max_attempts=5) -> list[ProfileView]:
    """Fetch all followers for a handle using cursor-based pagination from the AT Protocol client.
    Args:
        client (AsyncClient): An authenticated instance of the AT Protocol client.
        batch_limit (int, optional): The maximum number of followers to fetch per request. Defaults to 100.
        max_attempts (int, optional): The maximum number of retry attempts for failed requests.
    Returns:
        list: A list of follower records.
    """
    if isinstance(handle, ProfileView):
        handle = handle.handle
        
    @retry(
        wait=wait_exponential(multiplier=1, min=4, max=10),
        stop=stop_after_attempt(max_attempts),
        retry=retry_if_exception_type(Exception),
    )
    async def fetch_batch(cursor=None):
        response = await client.get_follows(
            actor=handle,
            limit=batch_limit,
            cursor=cursor
        )
        return response
    
    all_following = []
    cursor = None
    
    while True:
        response = await fetch_batch(cursor)
        all_following.extend(response.follows)
        
        # Check if there are more pages
        if not response.cursor:
            break
        cursor = response.cursor

    return all_following

following = await fetch_all_following("mantzarlis.com", client)
print(f"Total following fetched: {len(following)}")

Total following fetched: 1209


In [5]:
topic_response = await client.app.bsky.unspecced.get_trending_topics()
topics = topic_response.topics

In [6]:
topics[0].topic, topics[0].link

('Kristi Noem', '/profile/trending.bsky.app/feed/471339584')

In [7]:
from atproto_client.models.app.bsky.feed.search_posts import Params as SearchParams
from atproto_client.models.app.bsky.feed.get_feed import Params as GetParams

# Create params object with the feed URI
params = SearchParams(q=topics[0].topic)

# Retrieve posts from the specific feed using the feed URI
feed_response = await client.app.bsky.feed.search_posts(params=params)
validation = await client.app.bsky.feed.get_feed(GetParams(feed="at://did:plc:qrz3lhbyuxbeilrc6nekdqme/app.bsky.feed.generator/462435931"))

In [8]:
await client.app.bsky.feed.generator.list(repo=client.me.did)

ListRecordsResponse(records={}, cursor=None)

In [119]:
from atproto_client.models.app.bsky.feed.get_feed_generator import Params

ps = Params(feed="at://did:plc:qrz3lhbyuxbeilrc6nekdqme/app.bsky.feed.generator/462435931")

await client.app.bsky.feed.get_feed_generator(ps)

Response(is_online=True, is_valid=True, view=GeneratorView(cid='bafyreihyj6panjb7m3n47rartlwghz5k76zh7oehi73yaqibh6bvfzkhqq', creator=ProfileView(did='did:plc:qrz3lhbyuxbeilrc6nekdqme', handle='trending.bsky.app', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=None, feedgens=None, labeler=None, lists=None, starter_packs=None, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qrz3lhbyuxbeilrc6nekdqme/bafkreidovjqalz7xrudcz34g5uh74ffv4k57l5obxxqxykwlze2vbh7h6e@jpeg', created_at='2024-12-27T19:33:39.144Z', description=None, display_name='Bluesky Trending', indexed_at='2024-12-27T19:34:21.242Z', labels=[], status=None, verification=None, viewer=ViewerState(activity_subscription=None, blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=N

In [110]:
feed_response.posts

[PostView(author=ProfileViewBasic(did='did:plc:yt6gx2iwyhodrcqlfiytalqd', handle='spikes23.bsky.social', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=ProfileAssociatedChat(allow_incoming='none', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=None, labeler=None, lists=None, starter_packs=None, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:yt6gx2iwyhodrcqlfiytalqd/bafkreiduymeidfp7dnqzblx26kn5ed5ja2vzs6czdnonc4xobiqheenv2q@jpeg', created_at='2023-12-31T23:25:57.218Z', display_name='Ludo', labels=[Label(cts='1970-01-01T00:00:00.000Z', src='did:plc:yt6gx2iwyhodrcqlfiytalqd', uri='at://did:plc:yt6gx2iwyhodrcqlfiytalqd/app.bsky.actor.profile/self', val='!no-unauthenticated', cid='bafyreih6nr53eyn4zgbyhpmcmmwqwmoeolf2zrhdvimmn2kp4rtdbzbbha', exp=None, neg=None, sig=None,